# Welcome to Knuckledragger!

This tutorial can be used online at 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/philzook58/knuckledragger/blob/main/tutorial.ipynb)

The documentation for knuckledragger is available [here](https://www.philipzucker.com/knuckledragger/)

Running the following cell will install Knuckledragger on your colab instance.

In [ ]:
!git clone https://github.com/philzook58/knuckledragger.git

In [ ]:
!cd knuckledragger && python3 -m pip install -e .

You may need to restart your colab instance to get the installation to take. You can do so in the menu or by running the following cell.

In [ ]:
import os
os.kill(os.getpid(), 9)

# A Quick Tour

Knuckledragger is heavily built around the well used and appreciated python bindings to the SMT solver [Z3](https://microsoft.github.io/z3guide/). The terms and formulas are literally reexported Z3Py objects and there should be ZERO overhead interoperating any usage of Z3Py from other software into Knuckledragger. 

Anything Z3 can do on it's own, we can "prove" with no extra work

In [1]:
import kdrag as kd
import kdrag.smt as smt # smt is literally a reexporting of z3


p,q = smt.Bools("p q")
simple_taut = kd.prove(smt.Implies(p, smt.Or(p, q)))
simple_taut

|= Implies(p, Or(p, q))

The returned objects are `Proof`, not `smt.ExprRef` formulas

In [5]:
assert kd.kernel.is_proof(simple_taut)
assert not isinstance(simple_taut, smt.ExprRef)
type(simple_taut)

kdrag.kernel.Proof

In [7]:
simple_taut.thm

Implies(p, Or(p, q))

In [8]:
type(simple_taut.thm)

z3.z3.BoolRef

`kd.prove` will throw an error if the theorem is not provable

In [9]:
try:
    false_lemma = kd.prove(smt.Implies(p, smt.And(p, q)), timeout=10)
    print("This will not be reached")
except kd.kernel.LemmaError as _:
    pass

Z3 also supports things like Reals, Ints, BitVectors and strings

In [10]:
x = smt.Real("x")
real_trich = kd.prove(smt.ForAll([x], smt.Or(x < 0, x == 0, 0 < x)))
real_trich

|= ForAll(x, Or(x < 0, x == 0, x > 0))

In [11]:
x = smt.BitVec("x", 32)
or_idem = kd.prove(smt.ForAll([x], x | x == x))
or_idem

|= ForAll(x, x | x == x)

## Help Z3 Out

But the point of Knuckledragger is really for the things Z3 can't do in one shot. Knuckledragger supports a wrapped Z3 Boolean object called `kd.Proof` that only Knuckledragger knows how to build. This is how we distinguish between Z3 formulas that are _to be proven_ from those that _have been proven_. This `kd.Proof` object also retains a record of the previous calls to Z3, which can be considered a form of proof.

Knuckledragger support algebraic datatypes and induction

In [12]:
Nat = kd.Inductive("MyNat")
Zero = Nat.declare("Zero")
Succ = Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()

In [15]:
# We can define an addition function by cases
n,m = smt.Consts("n m", Nat)
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [n,m], 
    kd.cond(
        (n.is_Zero, m),
        (n.is_Succ, Nat.Succ(add(n.pred, m)))
))
add.defn

       add(n, m) ==
       If(is(Zero, n),
          m,
          If(is(Succ, n),
             Succ(add(pred(n), m)),
             unreachable!10))) to ForAll([n, m],
       add(n, m) ==
       If(is(Zero, n),
          m,
          If(is(Succ, n),
             Succ(add(pred(n), m)),
             unreachable!11)))


|= ForAll([n, m],
       add(n, m) ==
       If(is(Zero, n),
          m,
          If(is(Succ, n),
             Succ(add(pred(n), m)),
             unreachable!11)))

In [14]:
# There is a notation overloading mechanism modelled after python's singledispatch
kd.notation.add.register(Nat, add)
n + m

add(n, m)

In [ ]:
# The definitional lemma is not available to the solver unless you give it
add_zero_x = kd.prove(smt.ForAll([n], Nat.Zero + n == n), by=[add.defn])
add_succ_x = kd.prove(smt.ForAll([n,m], Nat.Succ(n) + m == Nat.Succ(n + m)), by=[add.defn])

In [16]:
# More involved proofs can be more easily done in an interactive tactic
# Under the hood, this boils down to calls to kd.prove
# These proofs are best understood by seeing the interactive output in a Jupyter notebook
l = kd.Lemma(smt.ForAll([n], n + Nat.Zero == n))

# [] ?|= ForAll(n, add(n, Zero) == n)
_n = l.fix()            

# [] ?|= add(n!0, Zero) == n!2213
l.induct(_n)              

# Case n!0 == Nat.Zero
# [] ?|= add(Zero, Zero) == Zero
l.auto(by=[add.defn])

# Case n!0 == Nat.Succ(n.pred)
# [] ?|= ForAll(a!0, Implies(add(a!0, Zero) == a!0, add(Succ(a!0), Zero) == Succ(a!0)))
l.auto(by=[add.defn])

# Finally the actual Proof is built
add_x_zero = l.qed()

## Axiomatic Theories

But we can also build our own sorts and axiomatic theories, like a theory of groups <https://en.wikipedia.org/wiki/Group_(mathematics)>. The following defines a signature of a group.

In [ ]:
G = smt.DeclareSort("G")
mul = smt.Function("mul", G, G, G)
e = smt.Const("e", G)
inv = smt.Function("inv", G, G)
kd.notation.mul.register(G, mul)

There are also some axioms a group satisfies.

In [ ]:
x, y, z = smt.Consts("x y z", G)
mul_assoc = kd.axiom(smt.ForAll([x, y, z], x * (y * z) == (x * y) * z))
id_left = kd.axiom(smt.ForAll([x], e * x == x))
inv_left = kd.axiom(smt.ForAll([x], inv(x) * x == e))

The right inverse law actually follows from this minimal set of axioms. This proof was constructed by following the wikipedia page.

In [ ]:
# The Calc tactic can allow one to write explicit equational proofs
c = kd.Calc([x], x * inv(x))
c.eq(e * (x * inv(x)), by=[id_left])
c.eq((inv(inv(x)) * inv(x)) * (x * inv(x)), by=[inv_left])
c.eq(inv(inv(x)) * ((inv(x) * x) * inv(x)), by=[mul_assoc])
c.eq(inv(inv(x)) * (e * inv(x)), by=[inv_left])
c.eq(inv(inv(x)) * inv(x), by=[id_left])
c.eq(e, by=[inv_left])
inv_right = c.qed()

# Intro to Z3

Knuckledragger is heavily based around the preexisting SMT solver Z3. Knowing about Z3 is useful even if you aren't interested 

You can find a tutorial notebook and video I made here https://github.com/philzook58/z3_tutorial


In [2]:
import z3

We then declare our variables, state a set of constraints we wish to hold, and then call the convenience function solve to get a solution.

In [3]:
x = z3.Int('x')
y = z3.Int('y')
z3.solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


We can also use z3 to prove properties and theorems.

In [4]:
p = z3.Bool("p")
my_true_thm = z3.Implies(p, p)
my_true_thm

Implies(p, p)

In [5]:
z3.prove(my_true_thm)

proved


If the property is not true, smt can supply a counterexample. 

In [6]:
q = z3.Bool("q")
my_false_thm = z3.Implies(q, p)
my_false_thm

Implies(q, p)

In [7]:
z3.prove(my_false_thm)

counterexample
[p = False, q = True]


# Why Knuckledragger?

Z3 is a tour de force. It's beautiful. However, it falls short of solving or verifying many kinds of problems we may be interested in.

1. It may in principle be able to solve something, but times out
2. Many mathematical and verification questions involve induction, which Z3 does not have strong built in support for. In general automating this is a tough problem
3. We may want to spell out what Z3 can do for use automatically, to make sure we understand
4. Z3 may not have good built in understanding of problem domains other systems like Mathematica and sympy do
5. A systematic approach to hide or abstract things z3 should not care about

For all these reasons, we may want a principled way to use Z3 to confirm 

In comes Knuckledragger.

In principle, Knuckledragger is designed to be extremely minimalist. It can be implemented as a small number of idioms. There is a library however.

1. Lots of little niceties. Pretty printing, useful combinators, tactics. The more easily you can write your statements, the more likely they are to correspond to what's in your head
2. A library of axiomatized theories
3. A bit of rigidity to guide you towards proofs that are more likely to be sound. 


In [12]:
def axiom(thm):
    return thm

def lemma(thm,by=[]):
    z3.prove(z3.Implies(z3.And(by), thm))
    return thm

1. Distinguishing between theorems to be proven and theorems that have been proven
2. It is convenient and builds confidence to record a trace of the result coming in through these functions. These traces are recorded in a `Proof` tree which exactly reflects the call tree to `lemma` and `axiom` that produced to the Proof.

In [11]:
from dataclasses import dataclass

@dataclass
class Proof:
    thm: z3.BoolRef
    reasons: list["Proof"]

def axiom(thm):
    return Proof(thm,[])

def lemma(thm,by=[]):
    assert all(isinstance(x,Proof) for x in by)
    z3.prove(z3.Implies(z3.And(by), thm))
    return Proof(thm,by)

This is in essence the contents of `knuckledragger.kernel`.

# Knuckledragger

But this has been packaged up for you in the `knuckledragger` package alongside many other goodies.

In [3]:
import kdrag as kd
import kdrag.smt as smt # z3 is re-exported as kdrag.smt
# or from kdrag.all import *

In [ ]:
G = smt.DeclareSort("G")
inv = smt.Function("inv", G, G)
mul = smt.Function("mul", G, G, G)
kd.notation.mul.register(G, mul) # enables e * inv(e) notation
e = smt.Const("e", G)
x,y,z = smt.Consts("x y z", G)
mul_assoc = kd.axiom(smt.ForAll([x,y,z], x * (y * z) == (x * y) * z))
mul_id = kd.axiom(smt.ForAll([x], x * e == x))
mul_inv = kd.axiom(smt.ForAll([x], x * inv(x) == e))

group_db = [mul_assoc, mul_id, mul_inv]
def glemma(thm,by=[]):
    return kd.prove(thm,by + group_db)

# External Solvers

Knuckledragger is designed to be used with other solvers. The experience using Z3 is by far the most polished due to the excellence of it's bindings. For some domain specific problems, or those requiring significant quantifier reasoning, the architecture of Z3 is not the best choice.

Some of the installed solvers include

- Vampire
- E Prover
- Twee
- nanoCoP-i
- Gappa
- and More!


In [ ]:
! cd knuckledragger && ./install.sh # get and build external solvers